In [71]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma


In [11]:
from dotenv import load_dotenv

load_dotenv()

True

In [9]:
import pandas as pd

books = pd.read_csv("books_cleaned.csv")


In [10]:
books


,Title,Authors,Description,Category,Publisher,Price Starting With ($),Publish Date (Month),Publish Date (Year)
0,Journey Through Heartsongs,"By Stepanek, Mattie J. T.",Collects poems written by the eleven-year-old ...,"Poetry , General",VSP Books,19.96,September,2001
1,In Search of Melancholy Baby,"By Aksyonov, Vassily, Heim, Michael Henry, and...",The Russian author offers an affectionate chro...,"Biography & Autobiography , General",Random House,4.99,June,1987
2,The Dieter's Guide to Weight Loss During Sex,"By Smith, Richard","A humor classic, this tongue-in-cheek diet pla...","Health & Fitness , Diet & Nutrition , Diets",Workman Publishing Company,4.99,January,1978
3,Germs : Biological Weapons and America's Secre...,"By Miller, Judith, Engelberg, Stephen, and Bro...","Deadly germs sprayed in shopping malls, bomb-l...","Technology & Engineering , Military Science",Simon & Schuster,4.99,October,2001
4,The Good Book: Reading the Bible with Mind and...,"By Gomes, Peter J.","""The Bible and the social and moral consequenc...","Religion , Biblical Biography , General",Harper Perennial,5.29,May,1998
...,...,...,...,...,...,...,...,...
57958,The Iraq Papers,"By Ehrenberg, John (EDT), McSherry, J. Patrice...",No foreign policy decision in recent history h...,"Political Science , American Government , Gen...",Oxford University Press,10.99,January,2010
57959,Like A Sister: A Novel,"By Daugharty, Janice",Sister cannot say exactly when or where she wa...,"Fiction , Literary",Harper,5.37,November,1999
57960,Creating Web Pages Simplified (3-D Visual Series),"By Maran, Ruth, Whitehead, Paul, and Marangrap...","An ""owner's manual"" for first-time Web page cr...","Computers , Internet , General",Hungry Minds Inc,5.95,January,1997
57961,EVA: The Real Key to Creating Wealth,"By Ehrbar, Al","Called ""today's hottest financial idea and get...","Business & Economics , Corporate Finance , Ge...",Wiley,29.96,October,1998


In [14]:
books = pd.read_csv("books_cleaned.csv")
#print("Available columns:", books.columns.tolist())

# Create a combined publish date column from the existing date columns
books["publish_date_combined"] = books["Publish Date (Month)"] + " " + books["Publish Date (Year)"].astype(str)

In [16]:
books["publish_date_combined"]

0        September 2001
1             June 1987
2          January 1978
3          October 2001
4              May 1998
              ...      
57958      January 2010
57959     November 1999
57960      January 1997
57961      October 1998
57962      January 1998
Name: publish_date_combined, Length: 57963, dtype: object

In [56]:
books["publish_date_combined"].to_csv("publish_date_combined.txt", sep = "\n",
index = False, header = False)

In [57]:
raw_documents = TextLoader("publish_date_combined.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=0, chunk_overlap=0, separator="\n")
documents = text_splitter.split_documents(raw_documents)

In [86]:
raw_documents

'September 2001\nJune 1987\nJanuary 1978\nOctober 2001\nMay 1998\nSeptember 1998\nApril 1992\nJanuary 1978\nMay 1999\nOctober 1994\nNovember 2003\nOctober 2002\nNovember 1994\nMarch 1998\nMarch 1995\nSeptember 1997\nNovember 1992\nJanuary 1982\nJanuary 1998\nMay 1998\nJanuary 1997\nOctober 1995\nJanuary 1994\nOctober 1995\nOctober 1997\nSeptember 1996\nOctober 1986\nJune 2003\nJune 2002\nJanuary 1995\nSeptember 2003\nFebruary 2003\nJanuary 1999\nAugust 1990\nSeptember 1995\nAugust 1991\nApril 1993\nFebruary 2002\nJanuary 1997\nMarch 2004\nFebruary 2001\nOctober 1991\nApril 1990\nJune 1993\nApril 2000\nNovember 1994\nMay 2001\nOctober 1988\nJanuary 1977\nNovember 2000\nJanuary 1996\nJuly 2001\nDecember 1997\nJune 1999\nOctober 1996\nApril 1993\nDecember 1991\nOctober 1988\nApril 2004\nFebruary 1996\nJuly 1995\nJune 1999\nJanuary 1990\nJanuary 1994\nMay 1976\nSeptember 1997\nJune 1983\nOctober 1992\nJuly 1995\nOctober 1994\nMay 1988\nJanuary 1992\nJune 1976\nNovember 1973\nApril 1989\nOc

In [91]:
pip install sentence-transformers

  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
   ---------------------------------------- 0.0/212.5 MB ? eta -:--:--
   ---------------------------------------- 0.8/212.5 MB 3.3 MB/s eta 0:01:04
    --------------------------------------- 3.4/212.5 MB 7.7 MB/s eta 0:00:28
   - -------------------------------------- 5.5/212.5 MB 8.0 MB/s eta 0:00:26
   - -------------------------------------- 6.8/212.5 MB 7.6 MB/s eta 0:00:27
   - -------------------------------------- 8.9/212.5 MB 7.9 MB/s eta 0:00:26
   -- ------------------------------------- 11.0/212.5 MB 8.3 MB/s eta 0:00:25
   -- ------------------------------------- 12.8/212.5 MB 8.3 MB/s eta 0:00:25
   -- ------------------------------------- 15.5/212.5 MB 8.8 MB/s eta 0:00:23
   --- ------------------------------------ 17.6/212.5 MB 9.1 MB/s eta 0:00:22
   --- ------------------------------------ 18.9/212.5 MB 8.6 MB/s eta 0:00:23
   ---- ----------------------------------- 21.5/212.5 MB 8.9 MB/s eta 0:00:22
   

In [94]:
from sentence_transformers import SentenceTransformer

# Initialize the sentence transformer model
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="all-MiniLM-L6-v2"
)

# Create the database with local embeddings


NameError: name 'embedding_functions' is not defined